## Lesson 07 — Pandas Part II: Data cleaning and wrangling

In this lesson we will cover some more advanced features of [Pandas](http://pandas.pydata.org).

### Readings

* [_Data Cleaning and Preparation_, by Wes McKinney](https://wesmckinney.com/book/data-cleaning)
* [_Data Wrangling: Join, Combine, and Reshape_, by Wes McKinney](https://wesmckinney.com/book/data-wrangling)
* [_SQL OUTER JOIN_, by IONOS Redaktion](https://www.ionos.de/digitalguide/hosting/hosting-technik/sql-outer-join/)

### Table of Contents

* [concat](#concat)
* [merge](#merge)
* [join](#join)
* [stack](#stack)
* [unstack](#unstack)
* [values](#values)
* [apply](#apply)
* [lambda](#lambda)
* [map](#map)
* [sort_index](#sort_index)
* [sort_values](#sort_values)
* [isnull](#isnull)
* [fillna](#fillna)

In [2]:
# import modules
import pandas as pd
import numpy as np

### Concatenating (appending) and merging (joining) DataFrames
See [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/merging.html) for more information.

This table describes how the funcitons were are going to learn about are related to each other:

Action        | Combine two          | Add one to another
--------------|----------------------|-------------------
Concatenating | pd.concat([df1, df2]) | df1.append(df2)
Merging       | pd.merge(df1, df2)   | df1.join(df2)

#### concat

See [concat documentation](https://pandas.pydata.org/docs/reference/api/pandas.concat.html) for more info.

```python
pd.concat(
    objs,
    axis=0,
    join='outer',
    ignore_index=False,
    keys=None,
    levels=None,
    names=None,
    verify_integrity=False,
    sort=False,
)
```

* `objs`: list or dict of Series, DataFrame, or Panel objects. If a dict is passed, the sorted keys will be used as the keys argument, unless it is passed, in which case the values will be selected (see below)
* `axis`: `{0/’index’, 1/’columns’}`, default `0`. The axis to concatenate along
* `join`: `{‘inner’, ‘outer’}`, default `‘outer’`. How to handle indexes on other axis (or axes). Outer for union and inner for intersection
* `ignore_index`: boolean, default `False`. If `True`, do not use the index values on the concatenation axis. The resulting axis will be labeled `0, ..., n - 1`. This is useful if you are concatenating objects where the concatenation axis does not have meaningful indexing information.
* `keys`: sequence, default `None`. Construct hierarchical index using the passed keys as the outermost level If multiple levels passed, should contain tuples.
* `levels`: list of sequences, default None. If keys passed, specific levels to use for the resulting MultiIndex. Otherwise they will be inferred from the keys.
* `names`: list, default `None`. Names for the levels in the resulting hierarchical index.
* `verify_integrity`: boolean, default False. Check whether the new concatenated axis contains duplicates. This can be very expensive relative to the actual data concatenation
* `sort`: Sort non-concatenation axis if it is not already aligned. One exception to this is when the non-concatentation axis is a DatetimeIndex and join=’outer’ and the axis is not already aligned. In that case, the non-concatenation axis is always sorted lexicographically.

In [3]:
df1 = pd.DataFrame({'A': ['A0', 'A1', 'A2', 'A3'],
                    'B': ['B0', 'B1', 'B2', 'B3'],
                    'C': ['C0', 'C1', 'C2', 'C3'],
                    'D': ['D0', 'D1', 'D2', 'D3']},
                    index=[0, 1, 2, 3])
df2 = pd.DataFrame({'A': ['A4', 'A5', 'A6', 'A7'],
                    'B': ['B4', 'B5', 'B6', 'B7'],
                    'C': ['C4', 'C5', 'C6', 'C7'],
                    'D': ['D4', 'D5', 'D6', 'D7']},
                    index=[4, 5, 6, 7]) 
df3 = pd.DataFrame({'A': ['A8', 'A9', 'A10', 'A11'],
                    'B': ['B8', 'B9', 'B10', 'B11'],
                    'C': ['C8', 'C9', 'C10', 'C11'],
                    'D': ['D8', 'D9', 'D10', 'D11']},
                    index=[8, 9, 10, 11])

In [4]:
df1

,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
2,A2,B2,C2,D2
3,A3,B3,C3,D3


In [5]:
df2

,A,B,C,D
4,A4,B4,C4,D4
5,A5,B5,C5,D5
6,A6,B6,C6,D6
7,A7,B7,C7,D7


In [6]:
df3

,A,B,C,D
8,A8,B8,C8,D8
9,A9,B9,C9,D9
10,A10,B10,C10,D10
11,A11,B11,C11,D11


In [9]:
frames = [df1, df2, df3]
result = pd.concat(frames)
result

,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
2,A2,B2,C2,D2
3,A3,B3,C3,D3
4,A4,B4,C4,D4
5,A5,B5,C5,D5
6,A6,B6,C6,D6
7,A7,B7,C7,D7
8,A8,B8,C8,D8
9,A9,B9,C9,D9


In [10]:
result = pd.concat(frames, keys=('x', 'y', 'z'))
result

A    B    C    D
x 0    A0   B0   C0   D0
  1    A1   B1   C1   D1
  2    A2   B2   C2   D2
  3    A3   B3   C3   D3
y 4    A4   B4   C4   D4
  5    A5   B5   C5   D5
  6    A6   B6   C6   D6
  7    A7   B7   C7   D7
z 8    A8   B8   C8   D8
  9    A9   B9   C9   D9
  10  A10  B10  C10  D10
  11  A11  B11  C11  D11

In [11]:
result.loc['y']

,A,B,C,D
4,A4,B4,C4,D4
5,A5,B5,C5,D5
6,A6,B6,C6,D6
7,A7,B7,C7,D7


In [12]:
df4 = pd.DataFrame({'B': ['B2', 'B3', 'B6', 'B7'],
                    'D': ['D2', 'D3', 'D6', 'D7'],
                    'F': ['F2', 'F3', 'F6', 'F7']},
                    index=[2, 3, 6, 7])
df4

,B,D,F
2,B2,D2,F2
3,B3,D3,F3
6,B6,D6,F6
7,B7,D7,F7


In [13]:
df1

,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
2,A2,B2,C2,D2
3,A3,B3,C3,D3


In [14]:
pd.concat([df1, df4]) # axis=0

,A,B,C,D,F
0,A0,B0,C0,D0,NaN
1,A1,B1,C1,D1,NaN
2,A2,B2,C2,D2,NaN
3,A3,B3,C3,D3,NaN
2,NaN,B2,NaN,D2,F2
3,NaN,B3,NaN,D3,F3
6,NaN,B6,NaN,D6,F6
7,NaN,B7,NaN,D7,F7


In [12]:
pd.concat([df1, df4], axis=1)

,A,B,C,D,B,D,F
0,A0,B0,C0,D0,NaN,NaN,NaN
1,A1,B1,C1,D1,NaN,NaN,NaN
2,A2,B2,C2,D2,B2,D2,F2
3,A3,B3,C3,D3,B3,D3,F3
6,NaN,NaN,NaN,NaN,B6,D6,F6
7,NaN,NaN,NaN,NaN,B7,D7,F7


In [15]:
pd.concat([df1, df4], axis=1, join='outer')

,A,B,C,D,B,D,F
0,A0,B0,C0,D0,NaN,NaN,NaN
1,A1,B1,C1,D1,NaN,NaN,NaN
2,A2,B2,C2,D2,B2,D2,F2
3,A3,B3,C3,D3,B3,D3,F3
6,NaN,NaN,NaN,NaN,B6,D6,F6
7,NaN,NaN,NaN,NaN,B7,D7,F7


In [16]:
pd.concat([df1, df4], axis=1, join='inner')

,A,B,C,D,B,D,F
2,A2,B2,C2,D2,B2,D2,F2
3,A3,B3,C3,D3,B3,D3,F3


In [28]:
# Not ignoring indexes (default)
pd.concat([df1, df4], ignore_index=False)

,A,B,C,D,F
0,A0,B0,C0,D0,NaN
1,A1,B1,C1,D1,NaN
2,A2,B2,C2,D2,NaN
3,A3,B3,C3,D3,NaN
2,NaN,B2,NaN,D2,F2
3,NaN,B3,NaN,D3,F3
6,NaN,B6,NaN,D6,F6
7,NaN,B7,NaN,D7,F7


In [29]:
# Ignoring indexes
pd.concat([df1, df4], ignore_index=True)

,A,B,C,D,F
0,A0,B0,C0,D0,NaN
1,A1,B1,C1,D1,NaN
2,A2,B2,C2,D2,NaN
3,A3,B3,C3,D3,NaN
4,NaN,B2,NaN,D2,F2
5,NaN,B3,NaN,D3,F3
6,NaN,B6,NaN,D6,F6
7,NaN,B7,NaN,D7,F7


In [30]:
s3 = pd.Series([0, 1, 2, 3], name='foo')
s4 = pd.Series([0, 1, 2, 3])
s5 = pd.Series([0, 1, 4, 5])
result = pd.concat([s3, s4, s5], axis=1)
result

,foo,0,1
0,0,0,0
1,1,1,1
2,2,2,4
3,3,3,5


In [31]:
result = pd.concat([s3, s4, s5], axis=1, keys=('red', 'blue', 'yellow'))
result

,red,blue,yellow
0,0,0,0
1,1,1,1
2,2,2,4
3,3,3,5


#### merge

See [merge documentation](https://pandas.pydata.org/docs/reference/api/pandas.merge.html) for more info.

```python
pd.merge(
    left, 
    right, 
    how='inner', 
    on=None, 
    left_on=None, 
    right_on=None, 
    left_index=False, 
    right_index=False,
    sort=False, 
    suffixes=('_x', '_y'), 
    copy=None, 
    indicator=False,
    validate=None,
)
```

* `left`: A DataFrame object.
* `right`: Another DataFrame object.
* `on`: Columns (names) to join on. Must be found in both the left and right DataFrame objects. If not passed and `left_index` and `right_index` are `False`, the intersection of the columns in the DataFrames will be inferred to be the join keys.
* `left_on`: Columns from the left DataFrame to use as keys. Can either be column names or arrays with length equal to the length of the DataFrame.
* `right_on`: Columns from the right DataFrame to use as keys. Can either be column names or arrays with length equal to the length of the DataFrame.
* `left_index`: If `True`, use the index (row labels) from the left DataFrame as its join key(s). In the case of a DataFrame with a MultiIndex (hierarchical), the number of levels must match the number of join keys from the right DataFrame.
* `right_index`: Same usage as `left_index` for the right DataFrame
* `how`: One of `'left'`, `'right'`, `'outer'`, `'inner'`. Defaults to inner. See below for more detailed description of each method
* `sort`: Defaults to `False`. Sort the result DataFrame by the join keys in lexicographical order. Otherwise, the order will depend on the join type.
* `suffixes`: A tuple of string suffixes to apply to overlapping columns. Defaults to ('_x', '_y').
* `copy`: Always copy data (default True) from the passed DataFrame objects, even when reindexing is not necessary. Cannot be avoided in many cases but may improve performance / memory usage. The cases where copying can be avoided are somewhat pathological but this option is provided nonetheless.
* `indicator`: Add a column to the output DataFrame called _merge with information on the source of each row. _merge is Categorical-type and takes on a value of left_only for observations whose merge key only appears in 'left' DataFrame, right_only for observations whose merge key only appears in 'right' DataFrame, and both if the observation’s merge key is found in both.
* `validate`: If specified, checks if merge is of specified type.
    * “one_to_one” or “1:1”: check if merge keys are unique in both left and right datasets.
    * “one_to_many” or “1:m”: check if merge keys are unique in left dataset.
    * “many_to_one” or “m:1”: check if merge keys are unique in right dataset.
    * “many_to_many” or “m:m”: allowed, but does not result in checks.



In [33]:
left = pd.DataFrame({'key': ['K0', 'K1', 'K2', 'K3'],
                     'A': ['A0', 'A1', 'A2', 'A3'],
                     'B': ['B0', 'B1', 'B2', 'B3']})
right = pd.DataFrame({'key': ['K3', 'K2', 'K1', 'K0'],
                      'C': ['C0', 'C1', 'C2', 'C3'],
                      'D': ['D0', 'D1', 'D2', 'D3']})

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


In [34]:
left

,key,A,B
0,K0,A0,B0
1,K1,A1,B1
2,K2,A2,B2
3,K3,A3,B3


In [35]:
right

,key,C,D
0,K3,C0,D0
1,K2,C1,D1
2,K1,C2,D2
3,K0,C3,D3


In [37]:
result = pd.merge(left, right, on='key')
result

,key,A,B,C,D
0,K0,A0,B0,C3,D3
1,K1,A1,B1,C2,D2
2,K2,A2,B2,C1,D1
3,K3,A3,B3,C0,D0


In [38]:
left = pd.DataFrame({'key1': ['K0', 'K0', 'K1', 'K2'],
                     'key2': ['K0', 'K1', 'K0', 'K1'],
                     'A': ['A0', 'A1', 'A2', 'A3'],
                     'B': ['B0', 'B1', 'B2', 'B3']})
right = pd.DataFrame({'key1': ['K0', 'K1', 'K1', 'K2'],
                      'key2': ['K0', 'K0', 'K0', 'K0'],
                      'C': ['C0', 'C1', 'C2', 'C3'],
                      'D': ['D0', 'D1', 'D2', 'D3']})

In [39]:
left

,key1,key2,A,B
0,K0,K0,A0,B0
1,K0,K1,A1,B1
2,K1,K0,A2,B2
3,K2,K1,A3,B3


In [40]:
right

,key1,key2,C,D
0,K0,K0,C0,D0
1,K1,K0,C1,D1
2,K1,K0,C2,D2
3,K2,K0,C3,D3


In [41]:
pd.merge(left, right, on=['key1', 'key2'])

,key1,key2,A,B,C,D
0,K0,K0,A0,B0,C0,D0
1,K1,K0,A2,B2,C1,D1
2,K1,K0,A2,B2,C2,D2


The `how` argument to merge specifies how to determine which keys are to be included in the resulting table. If a key combination does not appear in either the left or right tables, the values in the joined table will be NA. Here is a summary of the how options and their SQL equivalent names:


| Merge method    | SQL Join Name     | Description                               |
|-----------------|-------------------|-------------------------------------------|
| left            | LEFT OUTER JOIN   | Use keys from left frame only             |
| right	          | RIGHT OUTER JOIN	 | Use keys from right frame only            |
| outer	          | FULL OUTER JOIN	  | Use union of keys from both frames        |
| inner (default) | INNER JOIN	       | Use intersection of keys from both frames |

Here is a Venn Diagram visualization of the join types:

<center>
<img src="../images/ionos_joins.webp" width="60%"/>
</center>

In [45]:
pd.merge(left, right, how='left', on=['key1', 'key2'])

,key1,key2,A,B,C,D
0,K0,K0,A0,B0,C0,D0
1,K0,K1,A1,B1,NaN,NaN
2,K1,K0,A2,B2,C1,D1
3,K1,K0,A2,B2,C2,D2
4,K2,K1,A3,B3,NaN,NaN


In [46]:
result = pd.merge(left, right, how='right', on=['key1', 'key2'])
result

,key1,key2,A,B,C,D
0,K0,K0,A0,B0,C0,D0
1,K1,K0,A2,B2,C1,D1
2,K1,K0,A2,B2,C2,D2
3,K2,K0,NaN,NaN,C3,D3


In [47]:
result = pd.merge(left, right, how='outer', on=['key1', 'key2'])
result

,key1,key2,A,B,C,D
0,K0,K0,A0,B0,C0,D0
1,K0,K1,A1,B1,NaN,NaN
2,K1,K0,A2,B2,C1,D1
3,K1,K0,A2,B2,C2,D2
4,K2,K0,NaN,NaN,C3,D3
5,K2,K1,A3,B3,NaN,NaN


In [48]:
result = pd.merge(left, right, how='inner', on=['key1', 'key2'])
result

,key1,key2,A,B,C,D
0,K0,K0,A0,B0,C0,D0
1,K1,K0,A2,B2,C1,D1
2,K1,K0,A2,B2,C2,D2


In [49]:
# merge indicator
df1 = pd.DataFrame({'col1':[0,1], 'col_left':['a','b']})
df2 = pd.DataFrame({'col1':[1,2,2],'col_right':[2,2,2]})

In [34]:
df1

,col1,col_left
0,0,a
1,1,b


In [35]:
df2

,col1,col_right
0,1,2
1,2,2
2,2,2


In [36]:
# note: the argument indicator=True is an option with pandas 0.17.0 and greater
result = pd.merge(df1, df2, on='col1', how='outer', indicator='Merged')
result

,col1,col_left,col_right,Merged
0,0,a,NaN,left_only
1,1,b,2.0,both
2,2,NaN,2.0,right_only
3,2,NaN,2.0,right_only


#### join

The related DataFrame.join method, uses merge internally for the index-on-index and index-on-column(s) joins, **but joins on indexes by default** rather than trying to join on common columns (the default behavior for merge). If you are joining on index, you may wish to use DataFrame.join to save yourself some typing.

In [50]:
left = pd.DataFrame({'A': ['A0', 'A1', 'A2'],
                     'B': ['B0', 'B1', 'B2']},
                     index=['K0', 'K1', 'K2'])
right = pd.DataFrame({'C': ['C0', 'C2', 'C3'],
                      'D': ['D0', 'D2', 'D3']},
                      index=['K0', 'K2', 'K3'])

In [51]:
left

,A,B
K0,A0,B0
K1,A1,B1
K2,A2,B2


In [52]:
right

,C,D
K0,C0,D0
K2,C2,D2
K3,C3,D3


In [53]:
left.join(right, how='left')

,A,B,C,D
K0,A0,B0,C0,D0
K1,A1,B1,NaN,NaN
K2,A2,B2,C2,D2


In [54]:
left.join(right, how='right')

,A,B,C,D
K0,A0,B0,C0,D0
K2,A2,B2,C2,D2
K3,NaN,NaN,C3,D3


In [42]:
left.join(right, how='inner')

,A,B,C,D
K0,A0,B0,C0,D0
K2,A2,B2,C2,D2


In [43]:
left.join(right, how='outer')

,A,B,C,D
K0,A0,B0,C0,D0
K1,A1,B1,NaN,NaN
K2,A2,B2,C2,D2
K3,NaN,NaN,C3,D3


In [55]:
# overlapping value columns: suffixes
left = pd.DataFrame({'k': ['K0', 'K1', 'K2'], 'v': [1, 2, 3]})
right = pd.DataFrame({'k': ['K0', 'K0', 'K3'], 'v': [4, 5, 6]})

In [56]:
left

,k,v
0,K0,1
1,K1,2
2,K2,3


In [57]:
right

,k,v
0,K0,4
1,K0,5
2,K3,6


In [58]:
result = pd.merge(left, right, on='k')
result

,k,v_x,v_y
0,K0,1,4
1,K0,1,5


In [60]:
result = pd.merge(left, right, on='k', suffixes=['_l', '_r'])
result

,k,v_l,v_r
0,K0,1,4
1,K0,1,5


In [61]:
# Why would I use this? Example: Merging gene expression tables
degs = pd.DataFrame({'gene': ['cds1', 'cds3', 'cds6'], 'count': [345, 887, 459]})
names = pd.DataFrame({
    'gene': ['cds1', 'cds2', 'cds3', 'cds4', 'cds5', 'cds6'], 
    'description': ['primase', 'ligase', 'aldolase', 'amylase', 'polymerase', 'kinase']
})

In [62]:
degs

,gene,count
0,cds1,345
1,cds3,887
2,cds6,459


In [63]:
names

,gene,description
0,cds1,primase
1,cds2,ligase
2,cds3,aldolase
3,cds4,amylase
4,cds5,polymerase
5,cds6,kinase


In [64]:
degs_plus_names = pd.merge(degs, names)

In [65]:
degs_plus_names

,gene,count,description
0,cds1,345,primase
1,cds3,887,aldolase
2,cds6,459,kinase


In [66]:
# argument settings on and how were inferred/defaults
degs_plus_names = pd.merge(degs, names, on='gene', how='inner')

In [67]:
degs_plus_names

,gene,count,description
0,cds1,345,primase
1,cds3,887,aldolase
2,cds6,459,kinase


#### merge/join by index

In [68]:
degs.index = degs.gene
degs.drop('gene', axis=1, inplace=True)
degs

,count
gene,
cds1,345
cds3,887
cds6,459


In [70]:
names.index = names.gene
names.drop('gene', axis=1, inplace=True)
names

AttributeError: 'DataFrame' object has no attribute 'gene'

In [71]:
pd.merge(left=degs, right=names, left_index=True, right_index=True)

,count,description
gene,,
cds1,345,primase
cds3,887,aldolase
cds6,459,kinase


<a id="set_option"></a>

### World Series example

In [3]:
df_ws = pd.read_csv('../data/WorldSeriesWinners.txt', header=None)
df_ws.columns = ['team']

In [4]:
df_new = pd.DataFrame({'team': ['Atlanta Braves']})

In [5]:
df_new

,team
0,Atlanta Braves


#### pd.concat()

In [6]:
df2 = pd.concat([df_ws, df_new], ignore_index=True)
df2

,team
0,Boston Americans
1,No Winner
2,New York Giants
3,Chicago White Sox
4,Chicago Cubs
...,...
118,Atlanta Braves
119,Houston Astros
120,Texas Rangers
121,Los Angeles Dodgers


#### add columns

In [7]:
df2.shape[0]

123

In [8]:
df2['year'] = np.arange(1903, 1903 + df2.shape[0])
df2['first_initial'] = [team[0] for team in df2['team']]

In [22]:
df2

,team,year,first_initial
0,Boston Americans,1903,B
1,No Winner,1904,N
2,New York Giants,1905,N
3,Chicago White Sox,1906,C
4,Chicago Cubs,1907,C
...,...,...,...
118,Atlanta Braves,2021,A
119,Houston Astros,2022,H
120,Texas Rangers,2023,T
121,Los Angeles Dodgers,2024,L


#### stack

Data is often stored in so-called “stacked” or “record” format. In a “record” or “wide” format, typically there is one row for each subject. In the “stacked” or “long” format there are multiple rows for each subject where applicable.

“pivot” a level of the (possibly hierarchical) column labels, returning a DataFrame (or Series, when not having a multi-index) with an index with a new inner-most level of row labels.

<center>
<img src="../images/reshaping_stack.png" width="60%"/>
</center>

In [81]:
stack_df = df2.stack()
stack_df

0    team             Boston Americans
     year                         1903
     first_initial                   B
1    team                    No Winner
     year                         1904
                            ...       
121  year                         2024
     first_initial                   L
122  team               Atlanta Braves
     year                         2025
     first_initial                   A
Length: 369, dtype: object

In [79]:
type(stack_df)

pandas.core.series.Series

In [80]:
stack_df[113]

team             Chicago Cubs
year                     2016
first_initial               C
dtype: object

In [29]:
stack_df[113]["team"]

'Chicago Cubs'

In [86]:
stack_df.index

MultiIndex([(  0,          'team'),
            (  0,          'year'),
            (  0, 'first_initial'),
            (  1,          'team'),
            (  1,          'year'),
            (  1, 'first_initial'),
            (  2,          'team'),
            (  2,          'year'),
            (  2, 'first_initial'),
            (  3,          'team'),
            ...
            (119, 'first_initial'),
            (120,          'team'),
            (120,          'year'),
            (120, 'first_initial'),
            (121,          'team'),
            (121,          'year'),
            (121, 'first_initial'),
            (122,          'team'),
            (122,          'year'),
            (122, 'first_initial')],
           length=369)

In [39]:
stacked_df = pd.DataFrame(stack_df)
stacked_df

0
0   team           Boston Americans
    year                       1903
    first_initial                 B
1   team                  No Winner
    year                       1904
...                             ...
121 year                       2024
    first_initial                 L
122 team             Atlanta Braves
    year                       2025
    first_initial                 A

[369 rows x 1 columns]

#### unstack

(inverse operation of stack()) “pivot” a level of the (possibly hierarchical) row index to the column axis, producing a reshaped DataFrame with a new inner-most level of column labels.

<center>
<img src="../images/reshaping_unstack.png" width="60%"/>
</center>

In [89]:
unstack_df = df2.unstack()
unstack_df

team           0       Boston Americans
               1              No Winner
               2        New York Giants
               3      Chicago White Sox
               4           Chicago Cubs
                            ...        
first_initial  118                    A
               119                    H
               120                    T
               121                    L
               122                    A
Length: 369, dtype: object

In [90]:
type(unstack_df)

pandas.core.series.Series

In [91]:
unstack_df.index

MultiIndex([(         'team',   0),
            (         'team',   1),
            (         'team',   2),
            (         'team',   3),
            (         'team',   4),
            (         'team',   5),
            (         'team',   6),
            (         'team',   7),
            (         'team',   8),
            (         'team',   9),
            ...
            ('first_initial', 113),
            ('first_initial', 114),
            ('first_initial', 115),
            ('first_initial', 116),
            ('first_initial', 117),
            ('first_initial', 118),
            ('first_initial', 119),
            ('first_initial', 120),
            ('first_initial', 121),
            ('first_initial', 122)],
           length=369)

In [63]:
unstack_df["team"][113]

'Chicago Cubs'

In [99]:
pd.DataFrame(unstack_df)

0
team          0     Boston Americans
              1            No Winner
              2      New York Giants
              3    Chicago White Sox
              4         Chicago Cubs
...                              ...
first_initial 118                  A
              119                  H
              120                  T
              121                  L
              122                  A

[369 rows x 1 columns]

In [101]:
# unstacking a stacked series is the reverse operation
stack_df.unstack()

,team,year,first_initial
0,Boston Americans,1903,B
1,No Winner,1904,N
2,New York Giants,1905,N
3,Chicago White Sox,1906,C
4,Chicago Cubs,1907,C
...,...,...,...
118,Atlanta Braves,2021,A
119,Houston Astros,2022,H
120,Texas Rangers,2023,T
121,Los Angeles Dodgers,2024,L


### Survey data example with stack and unstack

In [126]:
# I have downloaded the World data properties from https://www.kaggle.com/datasets/nelgiriyewithana/countries-of-the-world-2023
df = pd.read_csv('../data/world-data-2023.csv', index_col=0)

In [127]:
df

,Density (P/Km2),Abbreviation,Agricultural Land( %),Land Area(Km2),Armed Forces size,Birth Rate,Calling Code,Capital/Major City,Co2-Emissions,CPI,...,Out of pocket health expenditure,Physicians per thousand,Population,Population: Labor force participation (%),Tax revenue (%),Total tax rate,Unemployment rate,Urban_population,Latitude,Longitude
Country,,,,,,,,,,,,,,,,,,,,,
Afghanistan,60,AF,58.10%,"652,230","323,000",32.49,93.0,Kabul,"8,672",149.9,...,78.40%,0.28,"38,041,754",48.90%,9.30%,71.40%,11.12%,"9,797,273",33.939110,67.709953
Albania,105,AL,43.10%,"28,748","9,000",11.78,355.0,Tirana,"4,536",119.05,...,56.90%,1.20,"2,854,191",55.70%,18.60%,36.60%,12.33%,"1,747,593",41.153332,20.168331
Algeria,18,DZ,17.40%,"2,381,741","317,000",24.28,213.0,Algiers,"150,006",151.36,...,28.10%,1.72,"43,053,054",41.20%,37.20%,66.10%,11.70%,"31,510,100",28.033886,1.659626
Andorra,164,AD,40.00%,468,NaN,7.20,376.0,Andorra la Vella,469,NaN,...,36.40%,3.33,"77,142",NaN,NaN,NaN,NaN,"67,873",42.506285,1.521801
Angola,26,AO,47.50%,"1,246,700","117,000",40.73,244.0,Luanda,"34,693",261.73,...,33.40%,0.21,"31,825,295",77.50%,9.20%,49.10%,6.89%,"21,061,025",-11.202692,17.873887
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Venezuela,32,VE,24.50%,"912,050","343,000",17.88,58.0,Caracas,"164,175","2,740.27",...,45.80%,1.92,"28,515,829",59.70%,NaN,73.30%,8.80%,"25,162,368",6.423750,-66.589730
Vietnam,314,VN,39.30%,"331,210","522,000",16.75,84.0,Hanoi,"192,668",163.52,...,43.50%,0.82,"96,462,106",77.40%,19.10%,37.60%,2.01%,"35,332,140",14.058324,108.277199
Yemen,56,YE,44.60%,"527,968","40,000",30.45,967.0,Sanaa,"10,609",157.58,...,81.00%,0.31,"29,161,922",38.00%,NaN,26.60%,12.91%,"10,869,523",15.552727,48.516388


In [128]:
# let's select a couple of markers for the dataset
df_subset = df[["Calling Code", "Capital/Major City", "Currency-Code", "Latitude", "Longitude", "Population"]]
df_subset

,Calling Code,Capital/Major City,Currency-Code,Latitude,Longitude,Population
Country,,,,,,
Afghanistan,93.0,Kabul,AFN,33.939110,67.709953,"38,041,754"
Albania,355.0,Tirana,ALL,41.153332,20.168331,"2,854,191"
Algeria,213.0,Algiers,DZD,28.033886,1.659626,"43,053,054"
Andorra,376.0,Andorra la Vella,EUR,42.506285,1.521801,"77,142"
Angola,244.0,Luanda,AOA,-11.202692,17.873887,"31,825,295"
...,...,...,...,...,...,...
Venezuela,58.0,Caracas,VED,6.423750,-66.589730,"28,515,829"
Vietnam,84.0,Hanoi,VND,14.058324,108.277199,"96,462,106"
Yemen,967.0,Sanaa,YER,15.552727,48.516388,"29,161,922"


In [129]:
df_transposed = df_subset.transpose()

In [130]:
df_transposed

Country,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,United States,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,Yemen,Zambia,Zimbabwe
Calling Code,93.0,355.0,213.0,376.0,244.0,1.0,54.0,374.0,61.0,43.0,...,44.0,1.0,598.0,998.0,678.0,58.0,84.0,967.0,260.0,263.0
Capital/Major City,Kabul,Tirana,Algiers,Andorra la Vella,Luanda,"St. John's, Saint John",Buenos Aires,Yerevan,Canberra,Vienna,...,London,"Washington, D.C.",Montevideo,Tashkent,Port Vila,Caracas,Hanoi,Sanaa,Lusaka,Harare
Currency-Code,AFN,ALL,DZD,EUR,AOA,XCD,ARS,AMD,AUD,EUR,...,GBP,USD,UYU,UZS,VUV,VED,VND,YER,ZMW,NaN
Latitude,33.93911,41.153332,28.033886,42.506285,-11.202692,17.060816,-38.416097,40.069099,-25.274398,47.516231,...,55.378051,37.09024,-32.522779,41.377491,-15.376706,6.42375,14.058324,15.552727,-13.133897,-19.015438
Longitude,67.709953,20.168331,1.659626,1.521801,17.873887,-61.796428,-63.616672,45.038189,133.775136,14.550072,...,-3.435973,-95.712891,-55.765835,64.585262,166.959158,-66.58973,108.277199,48.516388,27.849332,29.154857
Population,"38,041,754","2,854,191","43,053,054","77,142","31,825,295","97,118","44,938,712","2,957,731","25,766,605","8,877,067",...,"66,834,405","328,239,523","3,461,734","33,580,650","299,882","28,515,829","96,462,106","29,161,922","17,861,030","14,645,468"


In [132]:
df_subset.dtypes

Calling Code          float64
Capital/Major City     object
Currency-Code          object
Latitude              float64
Longitude             float64
Population             object
dtype: object

In [82]:
def score_to_numeric(score):
    converter = {'None': 0, 'Some': 1, 'Moderate': 2, 'Experienced': 3}
    return converter[score]

In [83]:
numeric_cols = ['score_command', 'score_r', 'score_matlab', 'score_perl', 'score_python']

In [84]:
for col in numeric_cols:
    df[col] = df[col].apply(score_to_numeric)

In [85]:
df

subject_id,computer_has,computer_os,score_command,score_r,score_matlab,score_perl,score_python
861167,Yes,Mac,1,1,1,0,1
346538,Yes,Windows;Unix,1,1,2,0,2
473138,Yes,Mac,2,0,3,0,2
268370,Yes,Mac,1,1,0,0,1
21801,Yes,Mac,2,1,0,0,1
...,...,...,...,...,...,...,...
189753,Yes,Mac;Windows;Unix,3,0,3,0,1
783121,Yes,Mac,0,1,0,0,0
474873,Yes,Mac;Unix,2,2,1,0,1
481858,Yes,Mac,1,2,0,0,1


In [86]:
df.dtypes

subject_id
computer_has     object
computer_os      object
score_command     int64
score_r           int64
score_matlab      int64
score_perl        int64
score_python      int64
dtype: object

In [87]:
# just select the numeric columns (exclude first 2 columns)
df.iloc[:,2:]

subject_id,score_command,score_r,score_matlab,score_perl,score_python
861167,1,1,1,0,1
346538,1,1,2,0,2
473138,2,0,3,0,2
268370,1,1,0,0,1
21801,2,1,0,0,1
...,...,...,...,...,...
189753,3,0,3,0,1
783121,0,1,0,0,0
474873,2,2,1,0,1
481858,1,2,0,0,1


#### df.stack() and df.unstack() (again)

In [88]:
df.iloc[:,2:].stack()

        subject_id   
861167  score_command    1
        score_r          1
        score_matlab     1
        score_perl       0
        score_python     1
                        ..
318480  score_command    1
        score_r          1
        score_matlab     0
        score_perl       0
        score_python     0
Length: 155, dtype: int64

In [89]:
df.iloc[:,2:].unstack()

subject_id           
score_command  861167    1
               346538    1
               473138    2
               268370    1
               21801     2
                        ..
score_python   189753    1
               783121    0
               474873    1
               481858    1
               318480    0
Length: 155, dtype: int64

<a id="values"></a>

#### values

In [90]:
# sometimes you need the values of a DataFrame, not the DataFrame representation of it

df.score_command

861167    1
346538    1
473138    2
268370    1
21801     2
         ..
189753    3
783121    0
474873    2
481858    1
318480    1
Name: score_command, Length: 31, dtype: int64

In [91]:
df.score_command.values

array([1, 1, 2, 1, 2, 1, 1, 1, 2, 2, 1, 0, 1, 2, 0, 1, 1, 1, 1, 1, 1, 0,
       0, 0, 1, 1, 3, 0, 2, 1, 1])

#### value_counts

In [92]:
df.score_command.value_counts()

1    18
2     6
0     6
3     1
Name: score_command, dtype: int64

<a id="apply"></a>
<a id="lambda"></a>

#### apply, lambda

In [93]:
f = lambda x: x.max() - x.min()

In [94]:
g = lambda x: x.value_counts()

In [95]:
df[numeric_cols].apply(f)

subject_id
score_command    3
score_r          3
score_matlab     3
score_perl       1
score_python     2
dtype: int64

In [96]:
df['score_command'].value_counts()

1    18
2     6
0     6
3     1
Name: score_command, dtype: int64

In [97]:
df[numeric_cols].apply(g)

subject_id,score_command,score_r,score_matlab,score_perl,score_python
0,6,12,15,28.0,11.0
1,18,12,7,3.0,13.0
2,6,6,4,NaN,7.0
3,1,1,5,NaN,NaN


<a id="map"></a>

#### map

In [98]:
df.dtypes

subject_id
computer_has     object
computer_os      object
score_command     int64
score_r           int64
score_matlab      int64
score_perl        int64
score_python      int64
dtype: object

In [99]:
# We can change the Y/N strings to boolean True/False using a dict and map
yn_tf = {'Yes': True, 'No': False}
df['computer_has'] = df['computer_has'].map(yn_tf)

In [100]:
df

subject_id,computer_has,computer_os,score_command,score_r,score_matlab,score_perl,score_python
861167,True,Mac,1,1,1,0,1
346538,True,Windows;Unix,1,1,2,0,2
473138,True,Mac,2,0,3,0,2
268370,True,Mac,1,1,0,0,1
21801,True,Mac,2,1,0,0,1
...,...,...,...,...,...,...,...
189753,True,Mac;Windows;Unix,3,0,3,0,1
783121,True,Mac,0,1,0,0,0
474873,True,Mac;Unix,2,2,1,0,1
481858,True,Mac,1,2,0,0,1


In [101]:
df.dtypes

subject_id
computer_has       bool
computer_os      object
score_command     int64
score_r           int64
score_matlab      int64
score_perl        int64
score_python      int64
dtype: object

<a id="sort_index"></a>
<a id="sort_values"></a>

#### sort_index, sort_values

In [102]:
df.sort_index(axis=0, ascending=True, inplace=False)

subject_id,computer_has,computer_os,score_command,score_r,score_matlab,score_perl,score_python
165235,True,Mac,0,0,0,0,0
189753,True,Mac;Windows;Unix,3,0,3,0,1
21801,True,Mac,2,1,0,0,1
232211,True,Mac,2,0,0,0,2
268370,True,Mac,1,1,0,0,1
...,...,...,...,...,...,...,...
861167,True,Mac,1,1,1,0,1
874782,True,Windows;Unix,2,1,1,0,2
944438,True,Mac,0,0,2,0,1
980220,True,Windows,1,2,0,1,0


In [103]:
df.sort_index(axis=1, ascending=True, inplace=False)

subject_id,computer_has,computer_os,score_command,score_matlab,score_perl,score_python,score_r
861167,True,Mac,1,1,0,1,1
346538,True,Windows;Unix,1,2,0,2,1
473138,True,Mac,2,3,0,2,0
268370,True,Mac,1,0,0,1,1
21801,True,Mac,2,0,0,1,1
...,...,...,...,...,...,...,...
189753,True,Mac;Windows;Unix,3,3,0,1,0
783121,True,Mac,0,0,0,0,1
474873,True,Mac;Unix,2,1,0,1,2
481858,True,Mac,1,0,0,1,2


In [104]:
df.sort_values('score_matlab', axis=0, ascending=False, inplace=False)

subject_id,computer_has,computer_os,score_command,score_r,score_matlab,score_perl,score_python
189753,True,Mac;Windows;Unix,3,0,3,0,1
473138,True,Mac,2,0,3,0,2
463808,True,Mac;Windows,1,1,3,0,1
442558,True,Windows,0,0,3,0,0
630741,True,Mac,0,0,3,0,1
...,...,...,...,...,...,...,...
47263,True,Mac,2,1,0,0,1
980220,True,Windows,1,2,0,1,0
21801,True,Mac,2,1,0,0,1
268370,True,Mac,1,1,0,0,1


In [105]:
df.sort_values(['score_matlab', 'score_r'], axis=0, ascending=False, inplace=False)

subject_id,computer_has,computer_os,score_command,score_r,score_matlab,score_perl,score_python
463808,True,Mac;Windows,1,1,3,0,1
473138,True,Mac,2,0,3,0,2
630741,True,Mac,0,0,3,0,1
442558,True,Windows,0,0,3,0,0
189753,True,Mac;Windows;Unix,3,0,3,0,1
...,...,...,...,...,...,...,...
783121,True,Mac,0,1,0,0,0
318480,True,Mac,1,1,0,0,0
232211,True,Mac,2,0,0,0,2
733358,True,Windows,1,0,0,0,1


In [106]:
df

subject_id,computer_has,computer_os,score_command,score_r,score_matlab,score_perl,score_python
861167,True,Mac,1,1,1,0,1
346538,True,Windows;Unix,1,1,2,0,2
473138,True,Mac,2,0,3,0,2
268370,True,Mac,1,1,0,0,1
21801,True,Mac,2,1,0,0,1
...,...,...,...,...,...,...,...
189753,True,Mac;Windows;Unix,3,0,3,0,1
783121,True,Mac,0,1,0,0,0
474873,True,Mac;Unix,2,2,1,0,1
481858,True,Mac,1,2,0,0,1


<a id="isnull"></a>
<a id="fillna"></a>

#### df.isnull(), df.fillna()

In [107]:
df['score_bash'] = np.nan

In [108]:
df

subject_id,computer_has,computer_os,score_command,score_r,score_matlab,score_perl,score_python,score_bash
861167,True,Mac,1,1,1,0,1,NaN
346538,True,Windows;Unix,1,1,2,0,2,NaN
473138,True,Mac,2,0,3,0,2,NaN
268370,True,Mac,1,1,0,0,1,NaN
21801,True,Mac,2,1,0,0,1,NaN
...,...,...,...,...,...,...,...,...
189753,True,Mac;Windows;Unix,3,0,3,0,1,NaN
783121,True,Mac,0,1,0,0,0,NaN
474873,True,Mac;Unix,2,2,1,0,1,NaN
481858,True,Mac,1,2,0,0,1,NaN


In [109]:
df.isnull()

subject_id,computer_has,computer_os,score_command,score_r,score_matlab,score_perl,score_python,score_bash
861167,False,False,False,False,False,False,False,True
346538,False,False,False,False,False,False,False,True
473138,False,False,False,False,False,False,False,True
268370,False,False,False,False,False,False,False,True
21801,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...
189753,False,False,False,False,False,False,False,True
783121,False,False,False,False,False,False,False,True
474873,False,False,False,False,False,False,False,True
481858,False,False,False,False,False,False,False,True


In [110]:
df.fillna(0, inplace=True)

In [111]:
df

subject_id,computer_has,computer_os,score_command,score_r,score_matlab,score_perl,score_python,score_bash
861167,True,Mac,1,1,1,0,1,0.0
346538,True,Windows;Unix,1,1,2,0,2,0.0
473138,True,Mac,2,0,3,0,2,0.0
268370,True,Mac,1,1,0,0,1,0.0
21801,True,Mac,2,1,0,0,1,0.0
...,...,...,...,...,...,...,...,...
189753,True,Mac;Windows;Unix,3,0,3,0,1,0.0
783121,True,Mac,0,1,0,0,0,0.0
474873,True,Mac;Unix,2,2,1,0,1,0.0
481858,True,Mac,1,2,0,0,1,0.0
